In [116]:
%matplotlib inline
import ephem
import matplotlib.pyplot as plt
import numpy as np

In [117]:
TLE = """Reaktor Hello World
1 43721U 18095A   18333.26388241 0.00000000  00000-0  00000-0 0  9992
2 43721  97.4900  38.2566 0011220 303.6883 257.4124 15.23034864    06"""

In [118]:
Sat = ephem.readtle(*TLE.split("\n"))

In [134]:
OUTPUT = """  Rises at {} in direction {}°
Maximum at {} at altitude {}°
   Sets at {} in direction {}°
    """

In [135]:
Obs = ephem.Observer()
Obs.lat = "60.217301"
Obs.lon = "24.394529"

In [141]:

def get_next_pass(min_alt=20):
    t0 = ephem.now()
    t = t0
    # Compute next pass from "now", and if it doesn't rise high enough,
    # add one second to its set time and compute next pass from that.
    # Stop computing after 14 days and just return an error message.
    while True:
        if t - t0 > 14:
            return "No pass found in the next 14 days."
        Obs.date = t
        Sat.compute(Obs)
        t_rise, az_rise, t_max, alt_max, t_set, az_set = Obs.next_pass(Sat)
        if alt_max > min_alt*np.pi/180:
            break
        t = t_set + 1 / 86400
    t_rise = ephem.Date(t_rise)
    t_max = ephem.Date(t_rise)
    t_set = ephem.Date(t_rise)
    return OUTPUT.format(
        ephem.localtime(t_rise),
        az_rise,
        ephem.localtime(t_max),
        alt_max,
        ephem.localtime(t_set),
        az_set
    )

## Get next pass that rises higher than given altitude

In [142]:
min_alt = 20
print(get_next_pass(min_alt))

  Rises at 2018-11-29 20:15:26.000003 in direction 127:15:23.7°
Maximum at 2018-11-29 20:15:26.000003 at altitude 25:08:56.9°
   Sets at 2018-11-29 20:15:26.000003 in direction 348:32:51.0°
    
